In [1]:
# Build a list of active users suitable for a Hive post
# Loads users from file users.txt. Lines starting with R are region and U are users

from datetime import datetime, timedelta, date
from beem import Hive
from beem.account import Account
from beem.amount import Amount
from beem.comment import Comment
from beem.exceptions import ContentDoesNotExistsException
from beem.instance import set_shared_blockchain_instance

In [2]:
hive = Hive()
set_shared_blockchain_instance(hive)
stop = datetime.utcnow() - timedelta(days=31) # Checks last month

In [3]:
def useractive(user):
    # Work back looking for last post/comment or vote
    acc = Account(user)#, blockchain_instance=hive)
    voted = commented = False

    for a in acc.history_reverse(stop=stop, only_ops=["vote", 
                                                      "comment"]):
        if a["type"] == "vote":
            voted = True
        if a["author"] == user:
            if a["type"] == "comment":
                commented = True
                break # Don't need to check if they voted as well
            
    return (voted, commented)

In [4]:
def output_region(region, active, voting, inactive):
    if region != None:
        act, voter, inact = len(active), len(voting), len(inactive)
        print(f'# {region} ({act}/{voter}/{inact})')
        if act:
            print('@'+ (', @'.join(sorted(active))))
            print()
        if voter:
            print('Voting: @'+ (', @'.join(sorted(voting))))
            print()
        if inact:
            print('Inactive: @'+ (', @'.join(sorted(inactive))))
            print()

In [5]:
with open('users.txt', 'r') as f:
    active = []
    inactive = []
    voting =[]
    region = None
    total_active = 0
    for l in f:
        #print(l)
        fields = l.split(',')
        if fields[0] == 'R':
            output_region(region, active, voting, inactive)
            total_active += len(active) # Will not count the last region, but exclude expats from total anyway
            region = fields[1].strip()
            active, voting, inactive = [], [], []
        else:
            name = dname = fields[0].strip()
            if '(' in name:
                dname = name
                name = name[:(name.find('(')-1)]
                
            v, c = useractive(name)
            if c:
                active.append(dname)
            elif v:
                voting.append(dname)
            else:
                inactive.append(dname)
    output_region(region, active, voting, inactive)
    print(f'**Total active** UK: {total_active}, Expats: {len(active)}')

# London & the South East (42/25/11)
@adetorrent, @andy4475, @artzo, @atomcollector (@risingstargame), @awah, @basilmarples, @blockchainyouth, @cezary-io, @chris-uk, @cryptogee, @dannyshine, @davidthompson57, @donatello, @fleur, @gillianpearce, @greatvideos, @imfarhad, @immarojas, @jedb, @kgakakillerg, @lloyddavis (@steemcampuk), @maneco64, @molometer, @nathanmars, @nickyhavey, @nicolcron, @notaboutme, @petrahaller, @raymondspeaks, @rmsbodybuilding, @robbieallenart, @sergiomendes, @spectrumecons, @stav, @steevc (@tenkminnows @swing47), @stickchumpion, @stickupboys, @stormkeepergu (@stormkeepersmdk), @teodora, @the-busy-bee, @ultravioletmag, @xxxxxxxxxxxxxxxx

Voting: @adamaslam, @bewithbreath, @bleepcoin, @charitycurator, @d3p0, @dashfit, @digna, @ezzy, @icepee, @kabir88, @kph, @liverehab, @markangeltrueman, @matildamoment, @mattniblock, @nakedverse, @nanzo-scoop, @pushpedal, @rafique, @redrica, @stimp1024, @tricki, @wilfredn, @winkandwoo, @zoltarian

Inactive: @blackrussian, @breadcen